In [1]:
import pandas as pd

train = pd.read_csv('../data/train/train.csv')
venue_item = pd.read_csv('../data/train/venue_item.csv')

In [2]:
# 주요 하이퍼파라미터
LOOKBACK, PREDICT = 28, 7 # 고정 (대회 규칙)
seed = 42

# 디바이스 설정
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Preprocess

In [3]:
# 음수 처리
train['매출수량'] = train['매출수량'].clip(lower=0)

In [4]:
# 업장, 메뉴 분리
sp = train["영업장명_메뉴명"].str.split("_", n=1, expand=True)
train["venue"] = sp[0]; train["item"] = sp[1]

In [5]:
# 날짜 형식 변환
train["영업일자"] = pd.to_datetime(train["영업일자"])

# 평일(월화수목)
train["is_weekday"] = train["영업일자"].dt.weekday < 4

# 고정일 공휴일(음력/대체공휴일 제외)
_fixed_mmdd = {"01-01","03-01","05-05","06-06","08-15","10-03","10-09","12-25"}
train["is_fixed_holiday"] = train["영업일자"].dt.strftime("%m-%d").isin(_fixed_mmdd)

In [6]:
# 카테고리, 특성 추가
train = train.merge(venue_item, left_on=['venue', 'item'], 
                   right_on=['venue', 'item'], how='left')

# 시간 특성 추가
train['year'] = train['영업일자'].dt.year
train['month'] = train['영업일자'].dt.month
train['day'] = train['영업일자'].dt.day
train['dayofweek'] = train['영업일자'].dt.dayofweek
train['quarter'] = train['영업일자'].dt.quarter
train['week_of_year'] = train['영업일자'].dt.isocalendar().week

# 계절 특성
train['season'] = train['month'].map({12:0, 1:0, 2:0,  # 겨울
                                     3:1, 4:1, 5:1,   # 봄  
                                     6:2, 7:2, 8:2,   # 여름
                                     9:3, 10:3, 11:3}) # 가을

## LightGBM

## CatBoost

## GRU

## Validation